# Constraining subglacial processes from surface velocity observations using surrogate-based Bayesian inference
## Part 1 - Training an ensemble of neural networks

In this notebook, we will illustrate the process of using Bayesian Bootstrap Aggregation (BayesBag) to train an ensemble of neural networks.  In this case, each ensemble member is one possible surrogate for the coupled hydrology-ice dynamics model described in the paper, mapping from a vector of 8 parameters to a velocity field.  We begin by importing both the parameters and the associated velocity fields computed by the physics model, which will act as training data for the surrogate.

In [ ]:
import pickle
import numpy as np
import re
#import utilities
import xarray as xr
from collections import OrderedDict

thin = 1

o_file = "../data/observed_speeds/greenland_vel_mosaic250_v1_g1800m.nc"
o_xr = xr.open_dataset(o_file)

o_speed = o_xr.variables["velsurf_mag"].values[::thin,::thin]
o_speed_sigma = o_xr.variables["velsurf_mag_error"].values[::thin,::thin]

o_ny, o_nx = o_speed.shape
o_xr.close()

o_speeds = np.nan_to_num(o_speed, 0).reshape(-1, 1)
o_log_speeds = np.log10(o_speeds)
o_log_speeds[np.isneginf(o_log_speeds)] = 0

o_speeds_sigma = np.nan_to_num(o_speed_sigma, 0).reshape(-1, 1)

from glob import glob
s_files = glob("../data/speeds_v2/velsurf_mag_gris_g1800m_v4_id_*.nc")
s_files = list(OrderedDict.fromkeys(s_files))

nt = len(s_files)

# open first file to get the dimensions
vxr0 = xr.open_dataset(s_files[0])
speed = vxr0.variables["velsurf_mag"].values[:,::thin,::thin]
_, ny, nx = speed.shape
vxr0.close()

m_speeds = np.zeros((nt, ny * nx))

# Go through all files and don't forget to extract the experiment id
ids = []
for k, s_file in enumerate(s_files):
    print(f"Reading {s_file}")
    vxr = xr.open_dataset(s_file)
    ids.append(re.search("id_(.+?)_0", s_file).group(1))
    m_speeds[k, ::] = vxr.variables["velsurf_mag"].values[:,::thin,::thin].flatten()
    vxr.close()

# For comparison, we run the analysis for both u and log10(u)
m_speeds_mean = np.nanmean(m_speeds, axis=0)
m_speeds_anom = m_speeds - m_speeds_mean
m_speeds_anom = np.nan_to_num(m_speeds_anom, 0)

m_log_speeds = np.log10(np.nan_to_num(m_speeds, 0))
m_log_speeds[np.isneginf(m_log_speeds)] = 0
m_log_speeds_mean = np.nanmean(m_log_speeds, axis=0)
m_log_speeds_anom =  m_log_speeds - m_log_speeds_mean
m_log_speeds_anom[np.isneginf(m_log_speeds_anom)] = 0
m_log_speeds_anom = np.nan_to_num(m_log_speeds_anom, 0)

In [ ]:
import pandas as pd
response_file = "log_speeds.csv"
df = pd.DataFrame(data=m_log_speeds, index=ids)
df.to_csv(response_file, index_label="id")

In [ ]:
from pismemulator.utils import prepare_data
import pandas as pd
response_file = "log_speeds.csv"
samples_file = "../data/samples/velocity_calibration_samples_100.csv"
samples, response = prepare_data(samples_file, response_file)


Preparing sample ../data/samples/velocity_calibration_samples_100.csv and response log_speeds.csv


The velocity fields have some bad simulations in them, so we filter out circumstances in which the model never ran past 12 years, and in which the max velocity was greater than 100km/a.

In [ ]:
print(samples.shape)
print(response.shape)

F = response.values
X = samples.values

In [ ]:
response

We will use pytorch to construct and train the neural networks.  To this end, we will move the physical model's parameters and (log-)speed fields to pytorch, and use the GPU if it's available.

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

m = X.shape[0]

X = torch.from_numpy(X)
F = torch.from_numpy(F)
F[F<0] = 0

X = X.to(torch.float32)
F = F.to(torch.float32)

X = X.to(device)
F = F.to(device)

X_m = X.mean(axis=0)
X_s = X.std(axis=0)

#F_m = F.mean(axis=0)
#F_s = F.std(axis=0)

X = (X-X_m)/X_s
#F = (F-F_m)/(F_s + 1e-5)

#X_hat = torch.log10(X)

In [ ]:
print(F.shape)

Part of our objective function is to weight by element area.  We will grab those values from a .vtu of an observed velocity field.

In [ ]:
import pickle
import numpy as np
#u_obs = utilities.VData('./data/u_observed.vtu')
#point_area = torch.tensor(u_obs.get_point_area(),dtype=torch.float,device=device)
normed_area = torch.tensor(np.ones(F.shape[1]),device=device)
normed_area/=normed_area.sum()
print(normed_area)

Next we need to define a few functions and classes.  First, we will create a function that extracts eigenglaciers and constructs the matrix $\hat{V}$, corresponding to the Dimensionality Reduction section.

In [ ]:
def get_eigenglaciers(omegas,F,cutoff=0.999):
    F_mean = (F*omegas).sum(axis=0)
    F_bar = F - F_mean # Eq. 28
    Z = torch.diag(torch.sqrt(omegas.squeeze()*m))
    U,S,V = torch.svd_lowrank(Z@F_bar,q=100)
    lamda = S**2/(m) 
    #print(S.shape)

    #lamda, V = torch.eig(S,eigenvectors=True) # Eq. 26
    #lamda = lamda[:,0].squeeze()
    
    cutoff_index = torch.sum(torch.cumsum(lamda/lamda.sum(),0)<cutoff)
    lamda_truncated = lamda.detach()[:cutoff_index]
    V = V.detach()[:,:cutoff_index]
    V_hat = V @ torch.diag(torch.sqrt(lamda_truncated)) # A slight departure from the paper: Vhat is the
                                                        # eigenvectors scaled by the eigenvalue size.  This
                                                        # has the effect of allowing the outputs of the neural
                                                        # network to be O(1).  Otherwise, it doesn't make 
                                                        # any difference.
    return V_hat, F_bar, F_mean

Second, we define the architecture of the neural network to be used as a surrogate.  This corresponds to the architecture defined in Fig. 3.

In [ ]:
import torch.nn as nn

class Emulator(nn.Module):
    def __init__(self,n_parameters,n_eigenglaciers,n_hidden_1,n_hidden_2,n_hidden_3,n_hidden_4,V_hat,F_mean):
        super().__init__()
        # Inputs to hidden layer linear transformation
        self.l_1 = nn.Linear(n_parameters, n_hidden_1)
        self.norm_1 = nn.LayerNorm(n_hidden_1)
        self.dropout_1 = nn.Dropout(p=0.0)
        self.l_2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.norm_2 = nn.LayerNorm(n_hidden_2)
        self.dropout_2 = nn.Dropout(p=0.5)
        self.l_3 = nn.Linear(n_hidden_2, n_hidden_3)
        self.norm_3 = nn.LayerNorm(n_hidden_3)
        self.dropout_3 = nn.Dropout(p=0.5)
        self.l_4 = nn.Linear(n_hidden_3, n_hidden_4)
        self.norm_4 = nn.LayerNorm(n_hidden_3)
        self.dropout_4 = nn.Dropout(p=0.5)
        self.l_5 = nn.Linear(n_hidden_4, n_eigenglaciers)

        self.V_hat = torch.nn.Parameter(V_hat,requires_grad=False)
        self.F_mean = torch.nn.Parameter(F_mean,requires_grad=False)

    def forward(self, x, add_mean=False):
        # Pass the input tensor through each of our operations

        a_1 = self.l_1(x)
        a_1 = self.norm_1(a_1)
        a_1 = self.dropout_1(a_1)
        z_1 = torch.relu(a_1) 

        a_2 = self.l_2(z_1)
        a_2 = self.norm_2(a_2)
        a_2 = self.dropout_2(a_2)
        z_2 = torch.relu(a_2) + z_1

        a_3 = self.l_3(z_2)
        a_3 = self.norm_3(a_3)
        a_3 = self.dropout_3(a_3)
        z_3 = torch.relu(a_3) + z_2
        
        a_4 = self.l_4(z_3)
        a_4 = self.norm_3(a_4)
        a_4 = self.dropout_3(a_4)
        z_4 = torch.relu(a_4) + z_3
        
        z_5 = self.l_5(z_4)
        if add_mean:
            F_pred = z_5 @ self.V_hat.T + self.F_mean
        else:
            F_pred = z_5 @ self.V_hat.T

        return F_pred 

In [ ]:
import torch.nn as nn

class EmulatorLearnableVhat(nn.Module):
    def __init__(self,n_parameters,n_eigenglaciers,n_hidden_1,n_hidden_2,n_hidden_3,n_hidden_4,n_grid_points,F_mean):
        super().__init__()
        # Inputs to hidden layer linear transformation
        self.l_1 = nn.Linear(n_parameters, n_hidden_1)
        self.norm_1 = nn.LayerNorm(n_hidden_1)
        self.dropout_1 = nn.Dropout(p=0.0)
        self.l_2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.norm_2 = nn.LayerNorm(n_hidden_2)
        self.dropout_2 = nn.Dropout(p=0.5)
        self.l_3 = nn.Linear(n_hidden_2, n_hidden_3)
        self.norm_3 = nn.LayerNorm(n_hidden_3)
        self.dropout_3 = nn.Dropout(p=0.5)
        self.l_4 = nn.Linear(n_hidden_3, n_hidden_4)
        self.norm_4 = nn.LayerNorm(n_hidden_3)
        self.dropout_4 = nn.Dropout(p=0.5)
        self.l_5 = nn.Linear(n_hidden_4, n_eigenglaciers)
        self.V_hat = nn.Linear(n_eigenglaciers,n_grid_points,bias=False)
        
        self.dropout_5 = nn.Dropout(p=0.3)

        #self.V_hat = torch.nn.Parameter(V_hat,requires_grad=False)
        self.F_mean = torch.nn.Parameter(F_mean,requires_grad=False)

    def forward(self, x, add_mean=False):
        # Pass the input tensor through each of our operations

        a_1 = self.l_1(x)
        a_1 = self.norm_1(a_1)
        a_1 = self.dropout_1(a_1)
        z_1 = torch.relu(a_1) 

        a_2 = self.l_2(z_1)
        a_2 = self.norm_2(a_2)
        a_2 = self.dropout_2(a_2)
        z_2 = torch.relu(a_2) + z_1

        a_3 = self.l_3(z_2)
        a_3 = self.norm_3(a_3)
        a_3 = self.dropout_3(a_3)
        z_3 = torch.relu(a_3) + z_2
        
        a_4 = self.l_4(z_3)
        a_4 = self.norm_3(a_4)
        a_4 = self.dropout_4(a_4)
        z_4 = torch.relu(a_4) + z_3
        
        z_5 = self.l_5(z_4)
        z_5 = self.dropout_5(z_5)
        z_6 = self.V_hat(z_5)
        if add_mean:
            F_pred = z_6 + self.F_mean
        else:
            F_pred = z_6
        return F_pred 

Third, we create an optimization procedure that trains a model for a given set of instance weights ($\omega_d$) and training data.  Optimization is performed using mini-batch gradient descent.

In [ ]:
from torch.utils.data import TensorDataset

def criterion_ae(F_pred,F_obs,omegas,area):
    instance_misfit = torch.sum(torch.abs((F_pred - F_obs))**2*area,axis=1)
    return torch.sum(instance_misfit*omegas.squeeze())

def train_surrogate(e,X_train,F_train,omegas,area,batch_size=128,epochs=3000,eta_0=0.01,k=1000.):
    
    omegas_0 = torch.ones_like(omegas)/len(omegas)
    training_data = TensorDataset(X_train,F_train,omegas)

    batch_size = 128
    train_loader = torch.utils.data.DataLoader(dataset=training_data,
                                               batch_size=batch_size, 
                                               shuffle=True)
    
    optimizer = torch.optim.Adam(e.parameters(),lr=eta_0,weight_decay=0.0)
    
    # Loop over the data
    for epoch in range(epochs):
        # Loop over each subset of data
        for param_group in optimizer.param_groups:
            param_group['lr'] = eta_0*(10**(-epoch/k))

        for x,f,o in train_loader:
            e.train()
            # Zero out the optimizer's gradient buffer
            optimizer.zero_grad()

            f_pred = e(x)

            # Compute the loss
            loss = criterion_ae(f_pred,f,o,area)

            # Use backpropagation to compute the derivative of the loss with respect to the parameters
            loss.backward()
        
            # Use the derivative information to update the parameters
            optimizer.step()
            
        e.eval()
        F_train_pred = e(X_train)
        # Make a prediction based on the model
        loss_train = criterion_ae(F_train_pred,F_train,omegas,area)
        # Make a prediction based on the model
        loss_test = criterion_ae(F_train_pred,F_train,omegas_0,area)

        # Print the epoch, the training loss, and the test set accuracy.
        if epoch%10==0:
            print(epoch,loss_train.item(),loss_test.item())

    

Here we put it all together: loop over the desired number of models, drawing random Bayesian bootstrap weights for each, training the surrogate, and saving the resulting models.  

In [ ]:
from scipy.stats import dirichlet

torch.manual_seed(0)
np.random.seed(0)

n_parameters = X.shape[1]
n_hidden_1 = 128
n_hidden_2 = 128
n_hidden_3 = 128
n_hidden_4 = 128

In [ ]:
n_models = 1 #To reproduce the paper, this should be 50
from scipy.stats import dirichlet
for model_index in range(n_models):
    omegas = torch.tensor(dirichlet.rvs(np.ones(m)),dtype=torch.float,device=device).T
   
    V_hat, F_bar, F_mean = get_eigenglaciers(omegas,F)
    n_eigenglaciers = V_hat.shape[1]
    n_grid_points = V_hat.shape[0]

    e = Emulator(n_parameters,n_eigenglaciers,n_hidden_1,n_hidden_2,n_hidden_3,n_hidden_4,V_hat,F_mean)
    e.to(device)
    
    train_surrogate(e,X,F_bar,omegas,normed_area,epochs=1000)
    torch.save(e.state_dict(),'emulator_ensemble_doug/emulator_{0:03d}.h5'.format(model_index))


In [ ]:
import matplotlib.pyplot as plt
F_pred = e(X,add_mean=True)
F_orig = F
fig,axs = plt.subplots(nrows=3,ncols=3)
fig.set_size_inches(16,30)
inds = np.random.randint(0,m,3)
axs[0,0].imshow(F_orig[inds[0],:].detach().cpu().numpy().reshape((ny,nx)),origin='lower',vmin=0,vmax=3)
axs[0,1].imshow(F_pred[inds[0],:].detach().cpu().numpy().reshape((ny,nx)),origin='lower',vmin=0,vmax=3)
axs[0,2].imshow((F_pred[inds[0],:].detach().cpu().numpy() - F_orig[inds[0],:].detach().cpu().numpy()).reshape((ny,nx)),origin='lower',vmin=-0.1,vmax=0.1)
axs[1,0].imshow(F_orig[inds[1],:].detach().cpu().numpy().reshape((ny,nx)),origin='lower',vmin=0,vmax=3)
axs[1,1].imshow(F_pred[inds[1],:].detach().cpu().numpy().reshape((ny,nx)),origin='lower',vmin=0,vmax=3)
axs[1,2].imshow((F_pred[inds[1],:].detach().cpu().numpy() - F_orig[inds[1],:].detach().cpu().numpy()).reshape((ny,nx)),origin='lower',vmin=-0.1,vmax=0.1)
axs[2,0].imshow(F_orig[inds[2],:].detach().cpu().numpy().reshape((ny,nx)),origin='lower',vmin=0,vmax=3)
axs[2,1].imshow(F_pred[inds[2],:].detach().cpu().numpy().reshape((ny,nx)),origin='lower',vmin=0,vmax=3)
axs[2,2].imshow((F_pred[inds[2],:].detach().cpu().numpy() - F_orig[inds[2],:].detach().cpu().numpy()).reshape((ny,nx)),origin='lower',vmin=-0.1,vmax=0.1)

from sklearn.metrics import mean_squared_error
for idx in range(3):
    rmsd = np.sqrt(mean_squared_error(10**F_pred[inds[idx], :].detach().cpu().numpy(), 10**F_orig[inds[0], :].detach().cpu().numpy()))
    print(rmsd)

In [ ]:
plt.imshow(e.V_hat.weight.detach().cpu().numpy()[:,19].reshape(ny,nx),origin='lower',vmin=-0.5,vmax=0.5)
plt.colorbar()

In [ ]:
nn.Linear?

## Part 2 - MCMC over the ensemble
Now that a number of neural network surrogates have been trained on random subsets of high-fidelity model runs, we will perform Markov Chain Monte Carlo sampling over each of these surrogates.  The correct parameter distribution for the high-fidelity model will be approximated by concatenating the Markov Chains over all of the surrogates.

In [ ]:
import pickle
import numpy as np
import torch


Read in the models trained above.

In [ ]:
models = []
n_models = 5 #To reproduce the paper, this should be 50

for i in range(n_models):
    state_dict = torch.load('emulator_ensemble_doug/emulator_{0:03d}.h5'.format(i))
    e = Emulator(state_dict['l_1.weight'].shape[1],state_dict['V_hat'].shape[1],n_hidden_1,n_hidden_2,n_hidden_3,n_hidden_4,state_dict['V_hat'],state_dict['F_mean'])
    e.load_state_dict(state_dict)
    e.to(device)
    e.eval()
    models.append(e)

Read in some relevant training data and ancillary values.  Convert observed velocities to speeds.

In [ ]:
#u_obs = utilities.VData('./data/u_observed.vtu')
#v_obs = utilities.VData('./data/v_observed.vtu')
#H_obs = utilities.VData('./data/H_observed.vtu')

#H = torch.tensor(H_obs.u)
#H = H.to(torch.float32).to(device)

U_obs = torch.tensor(np.nan_to_num(o_speed.ravel()))#((np.sqrt(u_obs.u**2 + v_obs.u**2))))
U_obs = U_obs.to(torch.float32).to(device)

Define the likelihood model, which requires a parameterization of observational uncertainty.

In [ ]:
from scipy.spatial.distance import pdist, squareform
#D = torch.tensor(squareform(pdist(u_obs.x)),dtype=torch.float32,device=device)

sigma2 = 10**2
#sigma_flow2 = 10**2
#alpha_cov = 1

l_model = 1e4#4*torch.sqrt(H.unsqueeze(1) @ H.unsqueeze(0))
#Sigma_obs = sigma2*torch.eye(U_obs.shape[0]*U_obs.shape[1],device=device)
#Sigma_flow = sigma_flow2*(1 + D**2/(2*alpha_cov*l_model**2))**-alpha_cov
#Sigma = Sigma_obs# + Sigma_flow

Construct the precision matrix (the inverse of equation 50)

In [ ]:
rho = 1./(1e4**2)
point_area = 81e6
K = point_area*rho

Tau = K * 1./sigma2 * K

In [ ]:
sigma_hat = np.sqrt(sigma2/K**2)

Construct the Beta prior distribution.  

In [ ]:
from scipy.stats import beta
alpha_b = 3.0
beta_b = 3.0

X_min = X.cpu().numpy().min(axis=0)-1e-3
X_max = X.cpu().numpy().max(axis=0)+1e-3

X_prior = beta.rvs(alpha_b,beta_b,size=(10000,X.shape[1]))*(X_max - X_min) + X_min

X_min = torch.tensor(X_min,dtype=torch.float32,device=device)
X_max = torch.tensor(X_max,dtype=torch.float32,device=device)

In [ ]:
X

This function returns a value that is proportional to the negative log-posterior distribution (The summands of equation 53).  

In [ ]:
alpha = 0.01
from scipy.special import gamma
nu = 1.
def V(X):
    U_pred = 10**m(X,add_mean=True)
    #print(U_pred.min())
    r = (U_pred - U_obs)
    X_bar = (X - X_min)/(X_max - X_min)

    #L1 = -0.5*Tau*r @ r
    #L1 = -(1 + r.shape[0])/2.*torch.log(1 + r@r*Tau)
    #L1 = torch.sum(-np.log(np.sqrt(np.pi*2)*sigma_hat) - 1./2.*(r/sigma_hat)**2)
    L1 = torch.sum(np.log(gamma((nu+1)/2.)) - np.log(gamma(nu/2.)) - np.log(np.sqrt(np.pi*nu)*sigma_hat) - (nu+1)/2.*torch.log(1 + 1./nu*(r/sigma_hat)**2))
    L2 = torch.sum((alpha_b-1)*torch.log(X_bar) + (beta_b-1)*torch.log(1-X_bar)) 

    #print(L1,L2)
    return -(alpha*L1 + L2)

In [ ]:
m = models[2]
X0 = torch.tensor(X_prior.mean(axis=0),dtype=torch.float,device=device)
print(X0)

We use the Metropolis-adjusted Langevin Algorithm to sample from the posterior distribution, which benefits from the availability of gradient and Hessian information.  Here, we compute these quantities (and some helpful additional ones) using automatic differentiation in pytorch.

In [ ]:
def get_log_like_gradient_and_hessian(V,X,eps=1e-2,compute_hessian=False):
    log_pi = V(X)
    if compute_hessian:
        g = torch.autograd.grad(log_pi,X,retain_graph=True,create_graph=True)[0]
        H = torch.stack([torch.autograd.grad(e,X,retain_graph=True)[0] for e in g])
        lamda,Q = torch.eig(H,eigenvectors=True)
        lamda_prime = torch.sqrt(lamda[:,0]**2 + eps)
        lamda_prime_inv = 1./torch.sqrt(lamda[:,0]**2 + eps)
        H = Q @ torch.diag(lamda_prime) @ Q.T
        Hinv = Q @ torch.diag(lamda_prime_inv) @ Q.T
        log_det_Hinv = torch.sum(torch.log(lamda_prime_inv))
        return log_pi,g,H,Hinv,log_det_Hinv
    else: 
        return log_pi

We initialize the sampler by first finding the Maximum A Posteriori parameter value, or MAP point.  We find the MAP point using gradient descent paired with a simple line search.

In [ ]:
def find_MAP(X,n_iters=50,print_interval=10):
    print('***********************************************')
    print('***********************************************')
    print('Finding MAP point')
    print('***********************************************')
    print('***********************************************')
    # Line search distances
    alphas = np.logspace(-4,0,11)
    # Find MAP point
    for i in range(n_iters):
        log_pi,g,H,Hinv,log_det_Hinv = get_log_like_gradient_and_hessian(V,X,compute_hessian=True)
        p = Hinv @ -g
        alpha_index = np.nanargmin([get_log_like_gradient_and_hessian(V,X + alpha*p,compute_hessian=False).detach().cpu().numpy() for alpha in alphas])
        mu = X + alphas[alpha_index] * p 
        X.data = mu.data
        if i%print_interval==0:
            print('===============================================')
            print('iter: {0:d}, ln(P): {1:6.1f}, curr. m: {2:4.4f},{3:4.2f},{4:4.2f},{5:4.2f},{6:4.2f},{7:4.2f},{8:4.2f},{9:4.2f}'.format(i,log_pi,*X.data.cpu().numpy()))
            print('===============================================')
    return X

With a good initial guess for the sampler discovered, we now implement the MALA algorithm.  

In [ ]:
def draw_sample(mu,cov,eps=1e-10):
    L = torch.cholesky(cov + eps*torch.eye(cov.shape[0],device=device))
    return mu + L @ torch.randn(L.shape[0],device=device)

def get_proposal_likelihood(Y,mu,inverse_cov,log_det_cov):
    return -0.5*log_det_cov - 0.5*(Y - mu) @ inverse_cov @ (Y-mu)

def MALA_step(X,h,local_data=None):
    if local_data is not None:
        pass  
    else:
        local_data = get_log_like_gradient_and_hessian(V,X,compute_hessian=True)
        
    log_pi,g,H,Hinv,log_det_Hinv = local_data
    
    X_ = draw_sample(X,2*h*Hinv).detach()
    X_.requires_grad=True
    
    log_pi_ = get_log_like_gradient_and_hessian(V,X_,compute_hessian=False)

    logq = get_proposal_likelihood(X_,X,H/(2*h),log_det_Hinv)
    logq_ = get_proposal_likelihood(X,X_,H/(2*h),log_det_Hinv)

    log_alpha = (-log_pi_ + logq_ + log_pi - logq)
    alpha = torch.exp(min(log_alpha,torch.tensor([0.],device=device)))
    u = torch.rand(1,device=device)
    if u <= alpha and log_alpha!=np.inf:
        X.data = X_.data
        local_data = get_log_like_gradient_and_hessian(V,X,compute_hessian=True)
        s = 1
    else:
        s = 0
    return X,local_data,s

def MALA(X,n_iters=10001,h=0.1,h_max=1.0,acc_target=0.25,k=0.01,beta=0.99,sample_path='./samples/',model_index=0,save_interval=1000,print_interval=50):
    print('***********************************************')
    print('***********************************************')
    print('Running Metropolis-Adjusted Langevin Algorithm for model index {0}'.format(model_index))
    print('***********************************************')
    print('***********************************************')
    local_data = None
    vars = []
    acc = acc_target
    for i in range(n_iters):
        X,local_data,s = MALA_step(X,h,local_data=local_data)
        vars.append(X.detach())
        acc = beta*acc + (1-beta)*s
        h = min(h*(1+k*np.sign(acc - acc_target)),h_max)
        if i%print_interval==0:
            print('===============================================')
            print('sample: {0:d}, acc. rate: {1:4.2f}, log(P): {2:6.1f}'.format(i,acc,local_data[0].item()))
            print('curr. m: {0:4.4f},{1:4.2f},{2:4.2f},{3:4.2f},{4:4.2f},{5:4.2f},{6:4.2f},{7:4.2f}'.format(*X.data.cpu().numpy()))
            print('===============================================')
          
        if i%save_interval==0:
            print('///////////////////////////////////////////////')
            print('Saving samples for model {0:03d}'.format(model_index))
            print('///////////////////////////////////////////////')
            X_posterior = torch.stack(vars).cpu().numpy()
            np.save(open(sample_path+'X_posterior_model_{0:03d}.npy'.format(model_index),'wb'),X_posterior)
    X_posterior = torch.stack(vars).cpu().numpy()
    return X_posterior       

In [ ]:
m = models[0]
X_0 = torch.tensor(X_prior.mean(axis=0),requires_grad=True,dtype=torch.float,device=device)
X_0 = find_MAP(X_0,n_iters=50)

In [ ]:
def V(m, X):
    U_pred = 10**m(X,add_mean=True)
    r = (U_pred - U_obs)
    X_bar = (X - X_min)/(X_max - X_min)


    L1 = torch.sum(np.log(gamma((nu+1)/2.)) - np.log(gamma(nu/2.)) - np.log(np.sqrt(np.pi*nu)*sigma_hat) - (nu+1)/2.*torch.log(1 + 1./nu*(r/sigma_hat)**2))
    L2 = torch.sum((alpha_b-1)*torch.log(X_bar) + (beta_b-1)*torch.log(1-X_bar)) 

    print(L1)
    return -(alpha*L1 + L2)

In [ ]:
U_pred = 10**m(X,add_mean=True)
r = (U_pred - U_obs)
print(f"r: {r}\n nu: {nu}\n sigma_hat: {sigma_hat}")

torch.log(1 + 1./nu*(r/sigma_hat)**2)

We now run the MAP/MALA procedure for each surrogate in the bootstrapped ensemble, and save the resulting posterior distributions.

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
for j,m in enumerate(models):
    X_0 = torch.tensor(X_prior.mean(axis=0),requires_grad=True,dtype=torch.float,device=device)
    X_0 = find_MAP(X_0,n_iters=50)
    # To reproduce the paper, n_iters should be 10^5
    X_posterior = MALA(X_0,n_iters=10000,model_index=j,sample_path="./posterior_samples_doug/", save_interval=1000,print_interval=100)

In [ ]:
models

In [ ]:
import matplotlib.pyplot as plt
fig,axs = plt.subplots(nrows=1,ncols=2)
fig.set_size_inches(12,24)
model_index = 1
X_posterior = np.load(open('./samples/X_posterior_model_{0:03d}.npy'.format(model_index),'rb'))
axs[0].imshow((m(torch.tensor(X_posterior[1000],device=device),add_mean=True) - m(torch.tensor(X_posterior[0],device=device),add_mean=True)).detach().cpu().numpy().reshape((ny,nx)),origin='lower',vmin=-0.1,vmax=0.1)
axs[1].imshow(np.log10(U_obs.detach().cpu().numpy().reshape((ny,nx))+1e-3),origin='lower',vmin=0,vmax=3)

In [ ]:
from matplotlib.ticker import NullFormatter,ScalarFormatter
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

#X_posterior = X_posterior*X_s.cpu().numpy() + X_m.cpu().numpy()
X_prior = X_prior*X_s.cpu().numpy() + X_m.cpu().numpy()

X_hat = X_prior

fig,axs = plt.subplots(nrows=8,ncols=8,figsize=(12,12))
X_list = []

for model_index in range(n_models):
    X_list.append(np.load(open('./posterior_samples_doug/X_posterior_model_{0:03d}.npy'.format(model_index),'rb')))
    
X_posterior = np.vstack(X_list)    
X_posterior = X_posterior*X_s.cpu().numpy() + X_m.cpu().numpy()

C_0 = np.corrcoef((X_posterior - X_posterior.mean(axis=0)).T)
Cn_0 = (np.sign(C_0)*C_0**2 + 1)/2.

color_post_0 = '#00B25F'
color_post_1 = '#132DD6'
color_prior = '#D81727'
color_ensemble = '#BA9B00'
color_other = '#20484E0'


for i in range(8):
    for j in range(8):
        if i>j:
            
            axs[i,j].scatter(X_posterior[:,j],X_posterior[:,i],c='k',s=0.5,alpha=0.05,label='Posterior',rasterized=True)
            min_val = min(X_hat[:,i].min(),X_posterior[:,i].min())
            max_val = max(X_hat[:,i].max(),X_posterior[:,i].max())
            bins_y = np.linspace(min_val,max_val,30)

            min_val = min(X_hat[:,j].min(),X_posterior[:,j].min())
            max_val = max(X_hat[:,j].max(),X_posterior[:,j].max())
            bins_x = np.linspace(min_val,max_val,30)
            
            #v = st.gaussian_kde(X_posterior[:,[j,i]].T)
            #bx = 0.5*(bins_x[1:] + bins_x[:-1])
            #by = 0.5*(bins_y[1:] + bins_y[:-1])
            #Bx,By = np.meshgrid(bx,by)
            
            #axs[i,j].contour(10**Bx,10**By,v(np.vstack((Bx.ravel(),By.ravel()))).reshape(Bx.shape),7,alpha=0.7,colors='black')

            axs[i,j].set_xlim(X_hat[:,j].min(),X_hat[:,j].max())
            axs[i,j].set_ylim(X_hat[:,i].min(),X_hat[:,i].max())

            #axs[i,j].set_xscale('log')
            #axs[i,j].set_yscale('log')

        elif i<j:
            patch_upper = Polygon(np.array([[0.,0.],[0.,1.],[1.,1.],[1.,0.]]),facecolor=plt.cm.seismic(Cn_0[i,j]))
            #patch_lower = Polygon(np.array([[0.,0.],[1.,0.],[1.,1.]]),facecolor=plt.cm.seismic(Cn_1[i,j]))
            axs[i,j].add_patch(patch_upper)
            #axs[i,j].add_patch(patch_lower)
            if C_0[i,j]>-0.5:
                color = 'black'
            else:
                color = 'white'
            axs[i,j].text(0.5,0.5,'{0:.2f}'.format(C_0[i,j]),fontsize=12,horizontalalignment='center',verticalalignment='center',transform=axs[i,j].transAxes,color=color)
            #if C_1[i,j]>-0.5:
            #    color = 'black'
            #else:
            #    color = 'white'

            #axs[i,j].text(0.75,0.25,'{0:.2f}'.format(C_1[i,j]),fontsize=12,horizontalalignment='center',verticalalignment='center',transform=axs[i,j].transAxes,color=color)

        elif i==j:
            min_val = min(X_hat[:,i].min(),X_posterior[:,i].min())
            max_val = max(X_hat[:,i].max(),X_posterior[:,i].max())
            bins = np.linspace(min_val,max_val,30)
            #X_hat_hist,b = np.histogram(X_hat[:,i],bins,density=True)
            X_prior_hist,b = np.histogram(X_prior[:,i],bins,density=True)
            X_posterior_hist = np.histogram(X_posterior[:,i],bins,density=True)[0]
            b = 0.5*(b[1:] + b[:-1])
            lw = 3.0
            axs[i,j].plot(b,X_prior_hist,color=color_prior,linewidth=0.5*lw,label='Prior',linestyle='dashed')
            
            axs[i,j].plot(b,X_posterior_hist,color='black',linewidth=lw,linestyle='solid',label='Posterior',alpha=0.7)

            #for X_ind in X_stack:
            #    X_hist,_ = np.histogram(X_ind[:,i],bins,density=False)
            #    X_hist=X_hist/len(X_posterior)
            #    X_hist=X_hist/(bins[1]-bins[0])
            #    axs[i,j].plot(10**b,X_hist,'b-',alpha=0.2,lw=0.5)

            if i==1:
                axs[i,j].legend(fontsize=8)
            axs[i,j].set_xlim(min_val,max_val)
            #axs[i,j].set_xscale('log')

        else:
            axs[i,j].remove()

keys=samples.keys()

for i,ax in enumerate(axs[:,0]):
    ax.set_ylabel(keys[i])

for j,ax in enumerate(axs[-1,:]):
    ax.set_xlabel(keys[j])
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    plt.setp(ax.xaxis.get_minorticklabels(), rotation=45)
    if j>0:
        ax.tick_params(axis='y',which='both',length=0)
        ax.yaxis.set_minor_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())

for ax in axs[:-1,1:].ravel():
    ax.xaxis.set_major_formatter(NullFormatter())
    ax.xaxis.set_minor_formatter(NullFormatter())
    ax.yaxis.set_major_formatter(NullFormatter())
    ax.yaxis.set_minor_formatter(NullFormatter())
    ax.tick_params(axis='both',which='both',length=0)

#fig.savefig('speed_emulator_posterior.pdf')

In [ ]:
from matplotlib.ticker import NullFormatter,ScalarFormatter
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

#X_posterior = X_posterior*X_s.cpu().numpy() + X_m.cpu().numpy()
#X_prior = X_prior*X_s.cpu().numpy() + X_m.cpu().numpy()

X_hat = X_prior

fig,axs = plt.subplots(nrows=8,ncols=8,figsize=(12,12))
color_post_0 = '#00B25F'
color_post_1 = '#132DD6'
color_prior = '#D81727'
color_ensemble = '#BA9B00'
color_other = '#20484E0'


for model_index in range(9):
    X_posterior = np.load(open('./samples/X_posterior_model_{0:03d}.npy'.format(model_index),'rb'))
    X_posterior = X_posterior*X_s.cpu().numpy() + X_m.cpu().numpy()
    
    
    C_0 = np.corrcoef((X_posterior - X_posterior.mean(axis=0)).T)
    Cn_0 = (np.sign(C_0)*C_0**2 + 1)/2.

    for i in range(8):
        for j in range(8):
            if i>j:
            
                axs[i,j].scatter(X_posterior[:,j],X_posterior[:,i],c='k',s=0.5,alpha=0.05,label='Posterior',rasterized=True)
                min_val = min(X_hat[:,i].min(),X_posterior[:,i].min())
                max_val = max(X_hat[:,i].max(),X_posterior[:,i].max())
                bins_y = np.linspace(min_val,max_val,30)

                min_val = min(X_hat[:,j].min(),X_posterior[:,j].min())
                max_val = max(X_hat[:,j].max(),X_posterior[:,j].max())
                bins_x = np.linspace(min_val,max_val,30)

                axs[i,j].set_xlim(X_hat[:,j].min(),X_hat[:,j].max())
                axs[i,j].set_ylim(X_hat[:,i].min(),X_hat[:,i].max())

            elif i<j:
                patch_upper = Polygon(np.array([[0.,0.],[0.,1.],[1.,1.],[1.,0.]]),facecolor=plt.cm.seismic(Cn_0[i,j]))
             
                axs[i,j].add_patch(patch_upper)
            
                if C_0[i,j]>-0.5:
                    color = 'black'
                else:
                    color = 'white'
                axs[i,j].text(0.5,0.5,'{0:.2f}'.format(C_0[i,j]),fontsize=12,horizontalalignment='center',verticalalignment='center',transform=axs[i,j].transAxes,color=color)

            elif i==j:
                min_val = min(X_hat[:,i].min(),X_posterior[:,i].min())
                max_val = max(X_hat[:,i].max(),X_posterior[:,i].max())
                bins = np.linspace(min_val,max_val,30)
                #X_hat_hist,b = np.histogram(X_hat[:,i],bins,density=True)
                X_prior_hist,b = np.histogram(X_prior[:,i],bins,density=True)
                X_posterior_hist = np.histogram(X_posterior[:,i],bins,density=True)[0]
                b = 0.5*(b[1:] + b[:-1])
                lw = 3.0
                axs[i,j].plot(b,X_prior_hist,color=color_prior,linewidth=0.5*lw,label='Prior',linestyle='dashed')
            
                axs[i,j].plot(b,X_posterior_hist,color='black',linewidth=lw,linestyle='solid',label='Posterior',alpha=0.7)

                #if i==1:
                #    axs[i,j].legend(fontsize=8)
                axs[i,j].set_xlim(min_val,max_val)
            else:
                axs[i,j].remove()

keys=samples.keys()

for i,ax in enumerate(axs[:,0]):
    ax.set_ylabel(keys[i])

for j,ax in enumerate(axs[-1,:]):
    ax.set_xlabel(keys[j])
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    plt.setp(ax.xaxis.get_minorticklabels(), rotation=45)
    if j>0:
        ax.tick_params(axis='y',which='both',length=0)
        ax.yaxis.set_minor_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())

for ax in axs[:-1,1:].ravel():
    ax.xaxis.set_major_formatter(NullFormatter())
    ax.xaxis.set_minor_formatter(NullFormatter())
    ax.yaxis.set_major_formatter(NullFormatter())
    ax.yaxis.set_minor_formatter(NullFormatter())
    ax.tick_params(axis='both',which='both',length=0)


In [ ]:
X_list = []

for model_index in range(n_models):
    X_list.append(np.load(open('./samples/X_posterior_model_{0:03d}.npy'.format(model_index),'rb')))
    
X_posterior = np.vstack(X_list)    
X_posterior = X_posterior*X_s.cpu().numpy() + X_m.cpu().numpy()

fig,axs = plt.subplots(nrows=3,ncols=3,figsize=(15,26))
axs = axs.ravel()
axs[0].imshow(np.log10(U_obs.detach().cpu().numpy().reshape((ny,nx))+1e-3),origin='lower',vmin=0,vmax=3)
axs[0].text(100,25,'Observed Speed\nVmin=0,Vmax=1000\nlog-scale',c='white')
for ax in axs[1:]:
    model_index = np.random.randint(10)
    sample_index = np.random.randint(9000)
    m = models[model_index]
    
    X_p = np.load(open('./samples/X_posterior_model_{0:03d}.npy'.format(model_index),'rb'))
    X_p_scaled = X_p*X_s.cpu().numpy() + X_m.cpu().numpy()   
    ax.imshow(m(torch.tensor(X_p[sample_index],device=device),add_mean=True).detach().cpu().numpy().reshape((ny,nx)),origin='lower',vmin=0,vmax=3)
    ax.text(120,25,'SIAE: {0:.2f}\nSSAN: {1:.2f}\nPPQ : {2:.2f}\nTEFO: {3:.2f}\nPHIM: {4:.2f}\nPHIX: {5:.2f}\nZMIN: {6:.2f}\nZMAX: {7:.2f}'.format(*X_p_scaled[sample_index]),c='white')
fig.subplots_adjust(wspace=0,hspace=0)
fig.savefig('emulator_speeds.pdf')

In [ ]:
U_obs.detach().cpu().numpy().reshape((ny,nx)).shape

In [ ]:
dataset.return_original().cpu().numpy().min(axis=0)

TODO: Add plotting